In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers
!pip install evaluate
!pip install f1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for f1: filename=f1-1.0.1-py3-none-any.whl size=1204 sha256=bb7b766a3fbca764cd6428f20314610812abdbbe4f0f999061cb182b74051437
  Stored in directory: /root/.cache/pip/wheels/5e/34/53/87317fe4016e96aa8f77d61d883fce7b7a5afcf558ed05eb05
Successfully built f1


In [2]:
import os
import evaluate 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast,AutoTokenizer,DataCollatorForTokenClassification,AutoModelForTokenClassification, TrainingArguments, Trainer,EarlyStoppingCallback
import json

In [3]:
path = os.path.join(os.getcwd(), "/kaggle/input/dataset")

with open(os.path.join(path,'train.json'), 'r') as f:
     train = json.loads(f.read())

with open(os.path.join(path,'test.json'), 'r') as f:
     test = json.loads(f.read())

In [4]:
print(train['text'][1])
print(train['index'][1])
print(train['POS'][1])
print(train['NER'][1])

['Polish', 'schoolgirl', 'blackmailer', 'wanted', 'textbooks', '.', 'GDANSK', ',', 'Poland', '1996-08-22', 'A', 'Polish', 'schoolgirl', 'blackmailed', 'two', 'women', 'with', 'anonymous', 'letters', 'threatening', 'death', 'and', 'later', 'explained', 'that', 'she', 'needed', 'money', 'for', 'textbooks', ',', 'police', 'said', 'on', 'Thursday', '.', '"', 'The', '13-year-old', 'girl', 'tried', 'to', 'extract', '60', 'and', '70', 'zlotys', '(', '$', '22', 'and', '$', '26', ')', 'from', 'two', 'residents', 'of', 'Sierakowice', 'by', 'threatening', 'to', 'take', 'their', 'lives', ',', '"', 'a', 'police', 'spokesman', 'said', 'in', 'the', 'nearby', 'northern', 'city', 'of', 'Gdansk', 'on', 'Thursday', '.', 'He', 'said', 'the', 'women', 'reported', 'the', 'blackmail', 'letters', 'and', 'police', 'caught', 'the', 'girl', 'on', 'Wednesday', 'as', 'she', 'tried', 'to', 'pick', 'up', 'the', 'cash', 'at', 'the', 'Sierakowice', 'railway', 'station', '.', '"', 'Interviewed', 'in', 'the', 'presence'

In [5]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
label_name = ['O', 'ORG', 'LOC', 'PER','MISC']
label_to_index = {y: x for x, y in enumerate(label_name)}
index_to_label = {x: y for x, y in enumerate(label_name)}

In [7]:
def indices_text_alignment(a_text_token,a_labels,a_indices):
  label_alignment = []
  index_alignment =[]
  for tokenized_text,labels,indices in zip(a_text_token,a_labels,a_indices):
      input_tokenized = tokenizer(tokenized_text, padding='max_length', max_length=512, truncation=True, is_split_into_words=True)
      id_of_word = input_tokenized.word_ids()
      token_labels_all = True
      prev_word_index = None
      id_index = []
      id_labels = []

      for index_word in id_of_word:
          if index_word is None:
            id_labels.append(-100)
            id_index.append(-100)
          elif index_word != prev_word_index:
            id_labels.append(label_to_index[labels[index_word]])
            id_index.append(indices[index_word])
          elif token_labels_all == True:
            id_index.append(indices[index_word]) 
            id_labels.append(label_to_index[labels[index_word]]) 
          else: 
            id_labels.append(-100)
            id_index.append(-100)
          
      prev_word_index = index_word

      label_alignment.append(id_labels)
      index_alignment.append(id_index)
  return {'labels': label_alignment,'indices': index_alignment}

In [9]:
train_text = [tokenizer(text,padding='max_length', max_length=512, truncation=True,is_split_into_words=True) for text in train['text']]
train_label_indices =  indices_text_alignment(train['text'], train['NER'],train['index'])
train_label_indices_dataframe = pd.DataFrame.from_dict(train_label_indices)

In [10]:
train_text_dataframe = pd.DataFrame.from_dict(train_text)
train_dataframe = train_text_dataframe.join(train_label_indices_dataframe)

In [11]:
train_dataframe.head()

,attention_mask,input_ids,token_type_ids,labels,indices
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 5726, 1352, 4788, 1177, 7666, 1107, 1340...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, ...","[-100, 0, 1, 2, 3, 3, 4, 5, 6, 7, 7, 7, 7, 7, ..."
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 3129, 1278, 17001, 1602, 14746, 1200, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, ...","[-100, 254, 255, 255, 256, 256, 256, 257, 258,..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2579, 1311, 3794, 3291, 3741, 2653, 3554...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, ...","[-100, 427, 428, 429, 430, 431, 432, 432, 433,..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1457, 20797, 1673, 117, 20497, 3923, 116...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, ...","[-100, 455, 455, 455, 456, 457, 457, 457, 458,..."
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1960, 5888, 1106, 2939, 1111, 3646, 1695...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, ...","[-100, 587, 588, 589, 590, 591, 592, 593, 594,..."


In [12]:
train_text_tokenized, validation_text_tokenized = train_test_split(train_dataframe, test_size=0.1, random_state=5)

In [13]:
print(train_text_tokenized.shape)
print(validation_text_tokenized.shape)

(680, 5)
(76, 5)


In [14]:
train_tokenized_dictionary = train_text_tokenized.to_dict('records')
validation_tokenized_dictionary = validation_text_tokenized.to_dict('records')

In [15]:

metric  = evaluate.load("f1")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
     
    true_predictions1 = [j for sub in true_predictions for j in sub]
    true_labels1 = [j for sub in true_labels for j in sub]

    results = metric.compute(predictions=true_predictions1, references=true_labels1,average="macro")
    print(results)
   
    return results

In [16]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=5)
training_args = TrainingArguments(output_dir='./log_results',
    num_train_epochs=20,
    learning_rate=2e-5,
    per_device_train_batch_size=16,   
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    warmup_steps=500, 
    eval_steps=50,
    save_steps=100,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dictionary,
    eval_dataset=validation_tokenized_dictionary,
    tokenizer=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer),
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 6)]
)

trainer.train()

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

Step,Training Loss,Validation Loss,F1
50,No log,1.141954,0.220286
100,No log,0.401819,0.500637
150,No log,0.173900,0.732488
200,No log,0.103008,0.886270
250,No log,0.081983,0.912976
300,No log,0.073315,0.925173
350,No log,0.070797,0.937318
400,No log,0.068796,0.941017


***** Running Evaluation *****
  Num examples = 76
  Batch size = 128
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: indices. If indices are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.


{'f1': 0.22028645983102318}


...

{'f1': 0.9373176936560743}


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 76
  Batch size = 128
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: indices. If indices are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.


{'f1': 0.9410170786195919}


Saving model checkpoint to ./log_results/checkpoint-400
Configuration saved in ./log_results/checkpoint-400/config.json
Model weights saved in ./log_results/checkpoint-400/pytorch_model.bin
tokenizer config file saved in ./log_results/checkpoint-400/tokenizer_config.json
Special tokens file saved in ./log_results/checkpoint-400/special_tokens_map.json
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./log_results/checkpoint-400 (score: 0.06879574060440063).


TrainOutput(global_step=440, training_loss=0.33890207464044747, metrics={'train_runtime': 696.9023, 'train_samples_per_second': 19.515, 'train_steps_per_second': 0.631, 'total_flos': 3553732276224000.0, 'train_loss': 0.33890207464044747, 'epoch': 20.0})

In [17]:
predictions = trainer.predict(validation_tokenized_dictionary)

***** Running Prediction *****
  Num examples = 76
  Batch size = 128
The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: indices. If indices are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.


{'f1': 0.9410170786195919}


In [19]:
def max_out(predictions):
  index_to_label= []
  for text in predictions:
    sen = []
    for label in text:
      sen.append(label_name[label.argmax()])
    index_to_label.append(sen)
  return index_to_label

In [20]:
validation_text_tokenized_dataframe = validation_text_tokenized
validation_text_tokenized_dataframe['Predicted_labels'] = max_out(predictions.predictions)
validation_text_tokenized_dataframe.head()

,attention_mask,input_ids,token_type_ids,labels,indices,Predicted_labels
84,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 12127, 5478, 1708, 1267, 1376, 2810, 110...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, ...","[-100, 17601, 17602, 17602, 17603, 17604, 1760...","[O, MISC, O, O, O, O, O, O, O, O, O, O, PER, P..."
513,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 156, 9244, 10954, 2069, 118, 12465, 1003...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 4, ...","[-100, 108947, 108947, 108947, 108947, 108948,...","[O, O, O, O, O, O, ORG, ORG, ORG, ORG, ORG, O,..."
12,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 156, 9244, 10954, 2069, 118, 26660, 1592...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 0, 0, ...","[-100, 2025, 2025, 2025, 2025, 2026, 2027, 202...","[O, O, O, O, O, O, MISC, MISC, MISC, MISC, MIS..."
753,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2124, 26422, 9171, 6628, 1103, 1411, 189...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 4, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, ...","[-100, 161789, 161790, 161791, 161792, 161793,...","[O, MISC, O, O, O, O, O, O, O, LOC, LOC, LOC, ..."
151,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 156, 9244, 10954, 2069, 118, 143, 16941,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, ...","[-100, 31505, 31505, 31505, 31505, 31506, 3150...","[O, O, O, O, O, O, MISC, MISC, MISC, MISC, O, ..."


In [21]:
def indices_alignment(tokenized_text,indices):
    input_tokenized = tokenizer(tokenized_text, padding='max_length', max_length=512, truncation=True, is_split_into_words=True)
    id_of_word = input_tokenized.word_ids()
    token_label_all = True
    prev_word_index = None
    id_index = []

    for index_word in id_of_word:

        if index_word is None:
            id_index.append(-100)

        elif index_word != prev_word_index:
                id_index.append(indices[index_word])

        elif token_label_all == True:
                id_index.append(indices[index_word]) 
  
        else:
          id_index.append(-100)

        prev_word_index = index_word

    return id_index

In [22]:
text_test_data = [tokenizer(text,padding='max_length', max_length=512, truncation=True,is_split_into_words=True) for text in test['text']]
indices_test = {'indices': [indices_alignment(i,j) for i,j in zip(test['text'], test['index'])]}

In [23]:
text_test_dataframe = pd.DataFrame.from_dict(text_test_data)
indices_test_dataframe = pd.DataFrame.from_dict(indices_test)
test_dataframe = text_test_dataframe.join(indices_test_dataframe)

In [24]:
test_dictionary = test_dataframe.to_dict('records')

In [25]:
test_predictions = trainer.predict(test_dictionary)

***** Running Prediction *****
  Num examples = 189
  Batch size = 128
The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: indices. If indices are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.


In [26]:

test_final_dataframe = test_dataframe
test_final_dataframe['predicted_labes'] = max_out(test_predictions.predictions)
test_final_dataframe.head()

,attention_mask,input_ids,token_type_ids,indices,predicted_labes
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 10163, 1105, 2870, 4010, 7270, 117, 1001...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, 9, ...","[O, LOC, O, LOC, O, ORG, O, ORG, O, O, LOC, LO..."
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 156, 9244, 10954, 2069, 118, 159, 2346, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 243, 243, 243, 243, 244, 245, 245, 245,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 156, 9244, 10954, 2069, 118, 26546, 2198...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 477, 477, 477, 477, 478, 479, 479, 479,...","[O, O, O, O, O, O, PER, PER, PER, PER, PER, O,..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1999, 17058, 25464, 1116, 12814, 129, 11...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 743, 744, 745, 745, 746, 747, 747, 747,...","[O, LOC, O, O, O, O, O, O, O, O, O, O, O, O, L..."
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 145, 9565, 12649, 26547, 19747, 11780, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 1004, 1004, 1004, 1005, 1005, 1005, 100...","[O, O, O, O, O, O, O, O, ORG, ORG, PER, PER, O..."


In [27]:
x = tokenizer.convert_ids_to_tokens(test_final_dataframe["input_ids"][0])
token_indices =[]
token_labels =[]
for ids,lables in zip(test_final_dataframe['indices'].to_numpy(),test_final_dataframe['predicted_labes'].to_numpy()):
  for i,j  in zip(ids,lables):
    if not(i in token_indices or i == -100):
        token_labels.append(j)
        token_indices.append(i)
   

In [28]:
def format_output_labels(token_labels, token_indices):
    """
    Returns a dictionary that has the labels (LOC, ORG, MISC or PER) as the keys, 
    with the associated value being the list of entities predicted to be of that key label. 
    Each entity is specified by its starting and ending position indicated in [token_indices].

    Eg. if [token_labels] = ["ORG", "ORG", "O", "O", "ORG"]
           [token_indices] = [15, 16, 17, 18, 19]
        then dictionary returned is 
        {'LOC': [], 'MISC': [], 'ORG': [(15, 16), (19, 19)], 'PER': []}

    :parameter token_labels: A list of token labels (eg. PER, LOC, ORG or MISC).
    :type token_labels: List[String]
    :parameter token_indices: A list of token indices (taken from the dataset) 
                              corresponding to the labels in [token_labels].
    :type token_indices: List[int]
    """
    label_dict = {"LOC":[], "MISC":[], "ORG":[], "PER":[]}
    prev_label = token_labels[0]
    start = token_indices[0]
    for idx, label in enumerate(token_labels):
      if prev_label != label:
        end = token_indices[idx-1]
        if prev_label != "O":
            label_dict[prev_label].append((start, end))
        start = token_indices[idx]
      prev_label = label
      if idx == len(token_labels) - 1:
        if prev_label != "O":
            label_dict[prev_label].append((start, token_indices[idx]))
    return label_dict

In [29]:
import numpy as np

def mean_f1(y_pred_dict, y_true_dict):
    """ 
    Calculates the entity-level mean F1 score given the actual/true and 
    predicted span labels.
    :parameter y_pred_dict: A dictionary containing predicted labels as keys and the 
                            list of associated span labels as the corresponding
                            values.
    :type y_pred_dict: Dict<key [String] : value List[Tuple]>
    :parameter y_true_dict: A dictionary containing true labels as keys and the 
                            list of associated span labels as the corresponding
                            values.
    :type y_true_dict: Dict<key [String] : value List[Tuple]>

    Implementation modified from original by author @shonenkov at
    https://www.kaggle.com/shonenkov/competition-metrics.
    """
    F1_lst = []
    for key in y_true_dict:
        TP, FN, FP = 0, 0, 0
        num_correct, num_true = 0, 0
        preds = y_pred_dict[key]
        trues = y_true_dict[key]
        for true in trues:
            num_true += 1
            if true in preds:
                num_correct += 1
            else:
                continue
        num_pred = len(preds)
        if num_true != 0:
            if num_pred != 0 and num_correct != 0:
                R = num_correct / num_true
                P = num_correct / num_pred
                F1 = 2*P*R / (P + R)
            else:
                F1 = 0      # either no predictions or no correct predictions
        else:
            continue
        F1_lst.append(F1)
    return np.mean(F1_lst)

In [30]:
import csv

def create_submission(output_filepath, token_labels, token_inds):
    """
    :parameter output_filepath: The full path (including file name) of the output file, 
                                with extension .csv
    :type output_filepath: [String]
    :parameter token_labels: A list of token labels (eg. PER, LOC, ORG or MISC).
    :type token_labels: List[String]
    :parameter token_indices: A list of token indices (taken from the dataset) 
                              corresponding to the labels in [token_labels].
    :type token_indices: List[int]
    """
    label_dict = format_output_labels(token_labels, token_inds)
    with open(output_filepath, mode='w') as csv_file:
        fieldnames = ['Id', 'Predicted']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for key in label_dict:
            p_string = " ".join([str(start)+"-"+str(end) for start,end in label_dict[key]])
            writer.writerow({'Id': key, 'Predicted': p_string})

In [31]:
import random
random.seed(43)

# test_pred_labels = ['PER', 'ORG', 'LOC', 'MISC']
# test_pred_inds = []

# for idx, ex in enumerate(test['text']):
#   for i, token in enumerate(ex):
#     test_pred_labels.append(random.choice(['PER', 'ORG', 'LOC', 'MISC', 'O']))
#   test_pred_inds += test['index'][idx]

# generate the file with predictions (the predicted_random.csv entry on kaggle)
create_submission('/kaggle/working/' + "/predicted_test_file_nmc210000.csv", token_labels, token_indices)